## Requirment Libraries

In [1]:
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import numpy as np

## Selenium Web Driver Setup

In [2]:
service = Service('C:/Users/fb_19/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')  # starts the browser in maximized window

options.add_argument('--disable-extensions')  # disables browser extensions
options.add_argument('--disable-gpu')  # disables hardware acceleration
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)


## Definition Of Element Lists

In [3]:

product_links = []
product_names = []
reviews = []
ids=[]
product_ratings=[]

## Reforming a Dataframe

In [4]:
df = pd.DataFrame(columns=("product-id","product-name","product-links","product-reviews","product-rating"))


## Collecting Product ID numbers

In [5]:
websites=("https://www.n11.com/telefon-ve-aksesuarlari?srt=REVIEWS&pg=","https://www.n11.com/agiz-dis-bakimi?srt=REVIEWS&pg=","https://www.n11.com/evcil-hayvan-urunleri?srt=REVIEWS&pg=")
for a in websites:
    for i in range(1, 15):
        try:
            website= a
            website += str(i)
            driver.get(website)
            website = a
            names = driver.find_elements(By.CLASS_NAME, 'productName')
            
            for name in names:
                product_names.append(name.text)
            # Wait for the columns to load
            wait = WebDriverWait(driver, 10)
            columns = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'column')))

            for column in columns:


                column_contents = column.find_elements(By.CLASS_NAME, 'columnContent')


                for content in column_contents:


                    id_element = content.find_element(By.TAG_NAME, "a")
                    id = id_element.get_attribute("data-id")
                    product_link = id_element.get_attribute("href")
                    if id is not None:

                        ids.append(id)
                        product_links.append(product_link)
                    else:
                        continue


        except TimeoutException:
            print("TimeoutException error ignored.")
            continue
            
print(len(ids))

1219


# Collecting Reviews

In [6]:
for a in ids:
    i=1
    website = "https://www.n11.com/component/render/groupProductReviews?page="+str(i)+"&productId="+str(a)
    driver.get(website)
    time.sleep(0.1)
    try:
        try:
            page= int(driver.find_element(By.XPATH, '//*[@id="reviewContent"]/div/div/span').text)
        except NoSuchElementException:
            page=1
        for i in range (1,page+1):
            website = "https://www.n11.com/component/render/groupProductReviews?page="+str(i)+"&productId="+str(a)
            driver.get(website)
            step1 = driver.find_elements(By.CLASS_NAME,'comment')
            for step in step1:
                try:
                    comment = step.find_element(By.TAG_NAME,'p').text
                    reviews.append(comment)
                    rating = step.find_element(By.CLASS_NAME,'commentTop').find_element(By.CLASS_NAME,'ratingCont').find_element(By.TAG_NAME,'span').get_attribute("class")
                    rating = int(rating.split("rating r")[1])/20
                    product_ratings.append(rating)
                    df.loc[len(df)]=[a,product_names[ids.index(a)],product_links[ids.index(a)],reviews[-1],product_ratings[-1]]
                except IndexError:
                    continue
    except TimeoutException:
        continue


In [7]:
len(reviews),len(product_ratings)

(200003, 199968)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199968 entries, 0 to 199967
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   product-id       199968 non-null  object 
 1   product-name     199968 non-null  object 
 2   product-links    199968 non-null  object 
 3   product-reviews  199968 non-null  object 
 4   product-rating   199968 non-null  float64
dtypes: float64(1), object(4)
memory usage: 9.2+ MB


In [12]:
df["product-reviews"].duplicated().value_counts()

product-reviews
False    179167
True      20801
Name: count, dtype: int64

In [13]:
df["product-rating"].value_counts()

product-rating
5.0    151496
4.0     19005
1.0     16502
3.0      8465
2.0      4500
Name: count, dtype: int64

In [29]:
df["product-reviews"]

0                                          🧿💙♥️🧡💛❤💚💙♥️👍🧡💛❤🧿
1         Siyah rengi aldım çok güzel bir rengi var 3 ay...
2                                   güzel ve kaliteli ürün.
3                                   güzel ve kaliteli ürün.
4                                   güzel ve kaliteli ürün.
                                ...                        
199963    gördüğüm en özenli paketleme ve hızlı kargo te...
199964    golden cinsi köpeğim çok severek yiyor, tüy dö...
199965                          Köpeğim bu mamaya bayılıyor
199966         Tavsiye ederim güzel paketlenmiş halde geldi
199967    Ürün çok iyi mutlaka köpeklerinize bu şekilde ...
Name: product-reviews, Length: 199968, dtype: object

In [35]:
data = df.copy()
data=data.drop_duplicates("product-reviews")

data.head()

,product-id,product-name,product-links,product-reviews,product-rating
0,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,🧿💙♥️🧡💛❤💚💙♥️👍🧡💛❤🧿,5.0
1,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,Siyah rengi aldım çok güzel bir rengi var 3 ay...,5.0
2,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,güzel ve kaliteli ürün.,5.0
6,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,iyi,5.0
7,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,orijinali gibi olmuyor tabiki biraz sert ama s...,3.0


In [36]:
data.to_csv('data-dropped-duplicated-reviews.csv')